In [ ]:
# %pip install scikit-learn
# %pip install tensorflow
# %pip install seaborn matplotlib pandas

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.backend import clear_session
from sklearn.model_selection import train_test_split
import shutil
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, Flatten
from tensorflow.keras.models import load_model
from tensorflow.keras import mixed_precision
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

# Enable mixed precision training
mixed_precision.set_global_policy('mixed_float16')

import warnings
warnings.filterwarnings('ignore')

clear_session()  # This frees the GPU memory, peventing training deadlock

# Function to create directories
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
for setClass in ['metal','glass', 'trash', 'cardboard', 'paper', 'plastic']:
    clear_session()
    global classez
    classez = [setClass, 'other']
    # data path
    global dataset_path
    dataset_path = f'/app/data/Datasets/augmented_{classez[0]}_and_{classez[1]}'  # <- docker file path      # "G:\My Drive\Datasets\ "

    tar = (224, 224)
    batch = 16 # 16
    #Split the dataset
    # Create an ImageDataGenerator for each subset
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # 20% for validation

    # Training data generator
    train_generator = datagen.flow_from_directory(
        dataset_path,
        target_size=tar,
        batch_size=batch,
        class_mode='binary',
        subset='training')  # Set as training data

    # Validation data generator
    validation_generator = datagen.flow_from_directory(
        dataset_path,
        target_size=tar,
        batch_size=batch,
        class_mode='binary',
        subset='validation')  # Set as validation data

    # Create a separate ImageDataGenerator for test data
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Test data generator
    test_generator = test_datagen.flow_from_directory(
        dataset_path,
        target_size=tar,
        batch_size=batch,
        class_mode='binary')  # No subset for testing
    
   

    # Paths to save splits
    base_dir = f'/app/data/Datasets/augmented_{classez[0]}_and_{classez[1]}-split'  # "G:\My Drive\Datasets\Trashnet-resized-split"
    train_dir = os.path.join(base_dir, 'train')
    val_dir = os.path.join(base_dir, 'val')
    test_dir = os.path.join(base_dir, 'test')

    create_dir(base_dir)
    create_dir(train_dir)
    create_dir(val_dir)
    create_dir(test_dir)

    for class_name in classez: #, 'cardboard', 'metal', 'paper', 'plastic', 'trash']:
        # Create class directories
        create_dir(os.path.join(train_dir, class_name))
        create_dir(os.path.join(val_dir, class_name))
        create_dir(os.path.join(test_dir, class_name))

        # List all files in the class directory
        class_files = os.listdir(os.path.join(dataset_path, class_name))
        train_files, test_files = train_test_split(class_files, test_size=0.2, random_state=42)  # 20% for testing
        train_files, val_files = train_test_split(train_files, test_size=0.25, random_state=42)  # 20% of remaining for validation (0.25 * 0.8 = 0.2)

        # Copy files to respective directories
        for file in train_files:
            shutil.copy(os.path.join(dataset_path, class_name, file), os.path.join(train_dir, class_name, file))

        for file in val_files:
            shutil.copy(os.path.join(dataset_path, class_name, file), os.path.join(val_dir, class_name, file))

        for file in test_files:
            shutil.copy(os.path.join(dataset_path, class_name, file), os.path.join(test_dir, class_name, file))
        
    train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=tar,
    batch_size=batch,
    class_mode='binary'
    # ,color_mode='grayscale'
    )

    validation_generator = datagen.flow_from_directory(
        val_dir,
        target_size=tar,
        batch_size=batch,
        class_mode='binary'
        # ,color_mode='grayscale'
        )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=tar,
        batch_size=batch,
        class_mode='binary'
        # ,color_mode='grayscale'
        )

    img_shape = (224,224,3)
    img_size = tar
    batch_size = batch #16 #32
    num_classes = 2
    log_file = '/app/data/code/accuracy_log7.csv'

    # DenseNet201 # MobileNetV3 # EfficientNetB0
    base_model = tf.keras.applications.DenseNet201(input_shape=img_shape,
                                                include_top=False,
                                                weights='imagenet')

    model_name = base_model.name #'EfficientNetB0' # 'DenseNet201' # 'MobileNetV2'
    # print(model_name)

    base_model.trainable = False
    for layer in base_model.layers[-6:]:
        layer.trainable = True
        
    class AccuracyLogger(Callback):
        def __init__(self,   log_file, model_name, img_shape, batch_size, num_classes):
            super().__init__()
            self.log_file = log_file
            self.model_name = model_name
            self.img_shape = img_shape
            self.batch_size = batch_size
            self.num_classes = num_classes
            
            # self.summary = summary

            # Write model and dataset information at the beginning of the log file
            with open(self.log_file, 'a') as f:
                f.write(f"\nModel: {self.model_name}\n")
                # f.write(f"Model summary:{self.summary}\n")
                f.write(f"Image shape: {self.img_shape}\n")
                f.write(f"Batch size: {self.batch_size}\n")
                f.write(f"Dataset: {dataset_path}\n")
                f.write(f"Number of classes: {self.num_classes}\n\n")
                f.write('Epoch,Accuracy,Validation Accuracy, Loss, Validation Loss\n')
                f.flush()

        def on_epoch_end(self, epoch, logs=None):
            logs = logs or {}
            accuracy = logs.get('accuracy')
            val_accuracy = logs.get('val_accuracy')
            loss = logs.get('loss')
            val_loss = logs.get('val_loss')

            # Write the epoch, accuracy, and validation accuracy to the log file
            with open(self.log_file, 'a') as f:
                f.write(f"{epoch + 1},{accuracy},{val_accuracy},{loss},{val_loss}\n")
                f.flush()

    model = tf.keras.Sequential([base_model,
                                #  tf.keras.layers.Flatten(),
                                tf.keras.layers.GlobalAveragePooling2D(),
                                #  tf.keras.layers.Dense(batch, activation="relu"),
                                tf.keras.layers.Dropout(0.2),
                                tf.keras.layers.Dense(1, activation="sigmoid")
                                #  tf.keras.layers.Dense(2, activation="softmax")
                                ])

    lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                    patience=2, 
                                    factor=0.3, #0.5,
                                    verbose=1, 
                                    min_lr=0.01) #0.01)
    model.compile(optimizer = 'sgd',
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])

    #Train for 10 epochs
    steps_per_epoch=int(len(train_generator)/batch_size)
    validation_steps = validation_generator.samples // validation_generator.batch_size

    # model.summary()
    # summary_buffer = io.StringIO()

    # # Redirect the stdout to the buffer and call model.summary()
    # with redirect_stdout(summary_buffer):
    #     model.summary()

    # # Get the summary as a string
    # summary = summary_buffer.getvalue()

    # Train the model
    history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        validation_data=validation_generator,
        callbacks=[AccuracyLogger(  log_file, model_name, img_shape, batch_size, num_classes),
                    EarlyStopping(patience=5)],
        epochs=30,
        verbose=1
    )

    def plot_graphs(history, batch_size):
        fig, axs = plt.subplots(1, 2, figsize=(12, 5))  # 1 row, 2 columns

        # Plot accuracy
        axs[0].plot(history.history['accuracy'])
        axs[0].plot(history.history['val_accuracy'])
        axs[0].set_title("Model Accuracy")
        axs[0].set_xlabel("Epochs")
        axs[0].set_ylabel("Accuracy")
        axs[0].legend(['Train', 'Validation'])

        # Plot loss
        axs[1].plot(history.history['loss'])
        axs[1].plot(history.history['val_loss'])
        axs[1].set_title("Model Loss")
        axs[1].set_xlabel("Epochs")
        axs[1].set_ylabel("Loss")
        axs[1].legend(['Train', 'Validation'])

        plt.tight_layout()  # Adjust layout to make sure plots don't overlap

        # Ensure the save directory exists
        save_dir = '/app/data/graphs/'
        os.makedirs(save_dir, exist_ok=True)

        # Create the filename using the dataset path and batch size
        dataset_name = os.path.basename(dataset_path[29:])  # filter out '/app/data/Datasets/' 
        filename = f"{dataset_name}_30epoch_batch{batch_size}_{model_name}.png"

        # Save the figure
        save_path = os.path.join(save_dir, filename)
        plt.savefig(save_path)
        print(f"Graph saved at: {save_path}")

        plt.show()

    # plot
    plot_graphs(history, batch)

    loss, accuracy = model.evaluate(test_generator, verbose=False)
    print("Testing Accuracy:  {:.4f}".format(accuracy))
    with open(log_file, 'a') as f:
        f.write(f"Testing Accuracy:  {accuracy:.4f}\n")
        f.flush()

    # save model
    model.save("tf_model.keras")

print("Training complete!")


Found 3662 images belonging to 2 classes.
Found 915 images belonging to 2 classes.
Found 4577 images belonging to 2 classes.
Found 2745 images belonging to 2 classes.
Found 916 images belonging to 2 classes.
Found 916 images belonging to 2 classes.


I0000 00:00:1726751128.017363      40 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726751128.054071      40 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726751128.054213      40 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726751128.055340      40 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step
Epoch 1/30


I0000 00:00:1726751147.494799     217 service.cc:146] XLA service 0x7b7b0c003240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726751147.494873     217 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1050 Ti, Compute Capability 6.1
2024-09-19 13:05:47.999169: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-19 13:05:50.094706: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
2024-09-19 13:06:02.312050: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.53GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


 1/10 ━━━━━━━━━━━━━━━━━━━━ 5:16 35s/step - accuracy: 0.3750 - loss: 0.8273

I0000 00:00:1726751172.938586     217 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


10/10 ━━━━━━━━━━━━━━━━━━━━ 68s 4s/step - accuracy: 0.5178 - loss: 0.7238 - val_accuracy: 0.6812 - val_loss: 0.5712
Epoch 2/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 835ms/step - accuracy: 0.5912 - loss: 0.6429 - val_accuracy: 0.8024 - val_loss: 0.4840
Epoch 3/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 836ms/step - accuracy: 0.7020 - loss: 0.5864 - val_accuracy: 0.6703 - val_loss: 0.5635
Epoch 4/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 837ms/step - accuracy: 0.7421 - loss: 0.5293 - val_accuracy: 0.8548 - val_loss: 0.4107
Epoch 5/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 836ms/step - accuracy: 0.7875 - loss: 0.4542 - val_accuracy: 0.8548 - val_loss: 0.3903
Epoch 6/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 840ms/step - accuracy: 0.7295 - loss: 0.5508 - val_accuracy: 0.7838 - val_loss: 0.4447
Epoch 7/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.7633 - loss: 0.5028